### Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

* tracking server: yes, local server
* backend store: sqlite database
* artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [5]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri='http://127.0.0.1:5000')

In [8]:
experiments = client.search_experiments()
for exp in experiments:
    print(exp)

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1733152015705, experiment_id='0', last_update_time=1733152015705, lifecycle_stage='active', name='Default', tags={}>


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/12/02 15:12:24 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/12/02 15:12:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/6748097c236c4b73804f72324676ea11/artifacts'
🏃 View run illustrious-foal-883 at: http://127.0.0.1:5000/#/experiments/1/runs/6748097c236c4b73804f72324676ea11
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [10]:
experiments = client.search_experiments()
for exp in experiments:
    print(exp)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1733152344217, experiment_id='1', last_update_time=1733152344217, lifecycle_stage='active', name='my-experiment-1', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1733152015705, experiment_id='0', last_update_time=1733152015705, lifecycle_stage='active', name='Default', tags={}>


### Interacting with the model registry

In [11]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [12]:
registered_models = client.search_registered_models()

In [14]:
from mlflow.tracking import MlflowClient
import mlflow

client = MlflowClient()

# Search for runs in the specified experiment
experiment_id = "1"
runs = client.search_runs(experiment_ids=[experiment_id], max_results=1)

if runs:
    run_id = runs[0].info.run_id  # Access the run_id
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name="iris-classifier"
    )
    print(f"Model registered with run_id: {run_id}")
else:
    print(f"No runs found for experiment_id: {experiment_id}")

Successfully registered model 'iris-classifier'.
2024/12/02 15:15:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1


Model registered with run_id: 6748097c236c4b73804f72324676ea11


Created version '1' of model 'iris-classifier'.
